# Data from  Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE) 
### https://github.com/CSSEGISandData/COVID-19

In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# import plotly.express as px
import plotly.graph_objects as go

import os 
import pandas as pd
import datetime

remote_dir = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'


In [44]:
def add_column_countries_provinces(df):
    fmt = lambda x: '' if x == 'nan' else ' - ' + x
    aux = df['Country/Region'].astype(str) + \
          df['Province/State'].astype(str).map(fmt)
    df['countries_provinces'] = aux
    return df 

def get_country_data(df, pattern, drop_cols=['Country/Region', 'Province/State', 'Lat', 'Long', 'countries_provinces']):
    ts = df[df['countries_provinces'].str.contains(pattern)]
    ts = ts.drop(drop_cols, axis=1)
    return ts

def get_country_dataframe(country):
    conf = get_country_data(confirmed, country)
    deat = get_country_data(deaths, country)
    reco = get_country_data(recovered, country)
    df = pd.concat({'Confirmed': conf, 'Deaths': deat, 'Recovered': reco})
    df = df.set_index(df.index.droplevel(1))
    df = df.T
    df['Active (C-R-D)'] = df['Confirmed'] - df['Recovered'] - df['Deaths']
    df['Active + Deaths (C-R)'] = df['Confirmed'] - df['Recovered']
#     df['Dates'] = df.index
    df.index = pd.to_datetime(df.index)
    return df

# get_country_dataframe('Portugal')

In [45]:
# read data

confirmed = pd.read_csv(os.path.join(remote_dir, 'time_series_covid19_confirmed_global.csv'))
deaths = pd.read_csv(os.path.join(remote_dir, 'time_series_covid19_deaths_global.csv'))
recovered = pd.read_csv(os.path.join(remote_dir, 'time_series_covid19_recovered_global.csv'))

# format data

confirmed = add_column_countries_provinces(confirmed)
deaths = add_column_countries_provinces(deaths)
recovered = add_column_countries_provinces(recovered)


In [47]:
df = get_country_dataframe('New Zealand')
df.tail(2)

,Confirmed,Deaths,Recovered,Active (C-R-D),Active + Deaths (C-R)
2020-04-25,1470,18,1142,310,328
2020-04-26,1469,19,1180,270,289


In [48]:

app = JupyterDash('CSSE Data')

app.layout = html.Div([
    
    dcc.Markdown('''
# COVID-19 Data

From [Johns Hopkins University Center for Systems Science and Engineering (JHU-CSSE)](https://github.com/CSSEGISandData/COVID-19)
'''),
    html.Label('Select country:'),
    dcc.Dropdown(
        id='dd-input-countries',
        options=[{'label': x, 'value': x} for x in confirmed['countries_provinces']],
        value='Portugal',
        style={'width':'60%'}
    ),
    html.Div(id='dd-output-graph')
])


@app.callback(
    dash.dependencies.Output('dd-output-graph', 'children'),
    [dash.dependencies.Input('dd-input-countries', 'value')])
def update_output(value):
    if value is not '':
        df = get_country_dataframe(value)
        data = []
        colors = ['blue', 'green', 'red']
        for i,yname in enumerate(['Confirmed', 'Recovered', 'Deaths']):
            data += [go.Scatter(x=df.index, y=df[yname],name=yname,
                               line=dict(color=colors[i]))]        

        yname = 'Active (C-R-D)'
        data += [go.Scatter(x=df.index, y=df[yname],name=yname,
                            line=dict(color='orange'))]

        yname = 'Active + Deaths (C-R)'
        data += [go.Scatter(x=df.index, y=df[yname],name=yname,
                            line=dict(color='orange', dash='dot'))]
        
        layout = dict(title=value, 
                      xaxis_title='Dates',
                      yaxis_title='Number of cases')
        
        fig = go.Figure(data=data, layout=layout)
        graph = dcc.Graph(figure=fig)
        return [graph]

app

In [ ]:
conf

In [ ]:
return